In [ ]:
import torch
from ttt.agents.az.training.trainer import AlphaZeroTrainer
from ttt.agents.az.training import load_config_from_yaml
from ttt.agents.az.net import masked_softmax
from ttt.env.state import TTTEnv
import torch.nn.functional as F

RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace prims; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:241; latest registration was registered at /dev/null:241

In [84]:
config_path = "ttt/ttt_config.yaml"
config = load_config_from_yaml(config_path)
trainer = AlphaZeroTrainer(config)
trainer.load_checkpoint('ttt/runs/run_20251030_112736/checkpoints/alphazero_epoch_25.pt')

Saved checkpoint: checkpoints\alphazero_epoch_0.pt
Loaded checkpoint from ttt/runs/run_20251030_112736/checkpoints/alphazero_epoch_25.pt


c:\Users\occoo\UTTT\ttt\agents\az\training\trainer.py:523: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.config.de

25

In [121]:
env = TTTEnv()
env.step(4)
env.step(6)
env.step(8)
env.step(1)
env.step(7)
obs_tensor = torch.from_numpy(env._encode()).unsqueeze(0).float().to('cuda')
net = trainer.network
logits, value = net.forward(obs_tensor)

legal_mask = env.legal_actions_mask()
legal_mask_tensor = torch.from_numpy(legal_mask.flatten()).unsqueeze(0).bool().to('cuda')

priors = F.softmax(logits, dim=-1) if legal_mask_tensor is None else masked_softmax(logits, legal_mask_tensor, dim=-1)
logits, priors, value

(tensor([[ 0.1809, -3.9389,  0.0949,  0.5206, -3.7663,  4.7562, -1.8865, -4.5252,
          -2.5813]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.0100, 0.0000, 0.0091, 0.0140, 0.0000, 0.9669, 0.0000, 0.0000, 0.0000]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.0090], device='cuda:0', grad_fn=<SqueezeBackward1>))

In [77]:
trainer_2 = AlphaZeroTrainer(config)
trainer_2.load_checkpoint('ttt/runs/run_20251029_213207/checkpoints/alphazero_epoch_0.pt')

env_2 = TTTEnv()
obs_tensor = torch.from_numpy(env_2._encode()).unsqueeze(0).float().to('cuda')
net_2 = trainer_2.network
logits, value = net_2.forward(obs_tensor)
legal_mask = env.legal_actions_mask()
legal_mask_tensor = torch.from_numpy(legal_mask.flatten()).unsqueeze(0).bool().to('cuda')

priors = F.softmax(logits, dim=-1) if legal_mask_tensor is None else masked_softmax(logits, legal_mask_tensor, dim=-1)
logits, priors, value

Saved checkpoint: checkpoints\alphazero_epoch_0.pt
Loaded checkpoint from ttt/runs/run_20251029_213207/checkpoints/alphazero_epoch_0.pt


(tensor([[ 0.2103,  0.1144, -0.6679, -0.2835, -0.1678, -0.6790,  0.4892, -0.4699,
          -0.7762]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.1605, 0.1458, 0.0667, 0.0979, 0.1099, 0.0659, 0.2121, 0.0813, 0.0598]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.7452], device='cuda:0', grad_fn=<SqueezeBackward1>))

In [1]:
import pandas as pd

In [7]:
import json
file_path = 'ttt/tournaments/series_AZ-Final_vs_RandomAgent_2025-11-04_14-02-21.json'

with open(file_path, 'r') as file:
    data = json.load(file)
data

{'meta': {'timestamp': '2025-11-04_14-02-21',
  'agent_A': 'AZ-Final',
  'agent_B': 'RandomAgent',
  'agent_A_id': 'AZ-Final',
  'agent_B_id': 'RandomAgent',
  'base_seed': 0,
  'instantiate_kwargs_A': {},
  'instantiate_kwargs_B': {},
  'schema': {'game': ['seed',
    'a_is_x',
    'winner',
    'moves',
    'action_probs',
    'priors',
    'moves_len'],
   'winner_values': {'X': 1, 'O': -1, 'draw': 0}}},
 'summary': {'n_games': 1000,
  'wins_A': 940,
  'wins_B': 0,
  'draws': 60,
  'avg_moves': 6.191},
 'games': [{'seed': 0,
   'a_is_x': True,
   'winner': 1,
   'moves': [4, 5, 0, 7, 6, 1, 3],
   'action_probs': [[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [],
    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    [],
    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
   'priors': [[0.20885310141066082,
     0.010478585310167609,
     0.0336896935263659,
     0.08685364063713288,
     0.2133472023411364,
     0.11644

In [6]:
x_wins = 0
x_draws = 0
o_wins = 0
o_draws = 0
for game in data['games']:
    if game['a_is_x']:
        if game['winner'] == 1:
            x_wins += 1
        else:
            x_draws += 1
    else:
        if game['winner'] == -1:
            o_wins += 1
        else:
            o_draws += 1
print(f'AlphaZero won as X {x_wins} times, and drew {x_draws} times.\nIt won as O {o_wins} times and drew {o_draws} times.')

AlphaZero won as X 495 times, and drew 5 times.
It won as O 445 times and drew 55 times.


In [8]:
rcs = 0
ercs = 0
for game in data['games']:
    if not game['a_is_x']:
        if game['moves'][0] == 4:
            rcs += 1
            if game['moves'][1] % 2 == 1:
                ercs += 1
print(f'Random went in the center as X {rcs} times, in which AlphaZero responded with an edge move {ercs} times')

Random went in the center as X 60 times, in which AlphaZero responded with an edge move 0 times


In [9]:
acs = 0
eacs = 0
wc = 0
for game in data['games']:
    if game['a_is_x']:
        if game['moves'][0] == 4:
            acs += 1
            if game['moves'][1] % 2 == 1:
                eacs += 1
                if game['winner'] == 1:
                    wc += 1
print(f'AlphaZero went in the center as X {acs} times, in which random responded with an edge move {eacs} times.\nAlphaZero won {wc} times from this position!')

AlphaZero went in the center as X 500 times, in which random responded with an edge move 226 times.
AlphaZero won 226 times from this position!


In [13]:
import json
file_path = 'uttt/tournaments/series_AZ-Epoch10_vs_HeuristicAgent_2025-11-03_10-56-06.json'

with open(file_path, 'r') as file:
    data = json.load(file)
data

{'meta': {'timestamp': '2025-11-03_10-56-06',
  'agent_A': 'AZ-Epoch10',
  'agent_B': 'HeuristicAgent',
  'agent_A_id': 'AZ-Epoch10',
  'agent_B_id': 'HeuristicAgent',
  'base_seed': None,
  'instantiate_kwargs_A': {},
  'instantiate_kwargs_B': {},
  'opening_book': [[40], [0], [8], [72], [80], [4], [36], [44], [76], [13]],
  'schema': {'game': ['seed', 'a_is_x', 'winner', 'moves', 'moves_len'],
   'winner_values': {'X': 1, 'O': -1, 'draw': 0}}},
 'summary': {'n_games': 20,
  'wins_A': 8,
  'wins_B': 7,
  'draws': 5,
  'avg_moves': 46.3},
 'games': [{'seed': None,
   'a_is_x': False,
   'winner': 1,
   'moves': [40,
    32,
    16,
    50,
    78,
    56,
    26,
    62,
    6,
    20,
    60,
    18,
    74,
    71,
    51,
    73,
    75,
    65,
    42,
    37,
    39,
    45,
    72,
    63,
    28,
    5,
    41,
    43,
    69,
    36,
    38,
    52,
    76,
    67,
    33],
   'action_probs': [],
   'priors': [],
   'moves_len': 35},
  {'seed': None,
   'a_is_x': True,
   'winn

In [14]:
x_wins = 0
x_draws = 0
x_losses = 0
o_wins = 0
o_draws = 0
o_losses = 0
for game in data['games']:
    if game['a_is_x']:
        if game['winner'] == 1:
            x_wins += 1
        elif game['winner'] == 0:
            x_draws += 1
        else:
            x_losses += 1
    else:
        if game['winner'] == -1:
            o_wins += 1
        elif game['winner'] == 0:
            o_draws += 1
        else:
            o_losses += 1

print(f'AlphaZero won as X {x_wins} times, drew {x_draws} times, and lost {x_losses} times.\nIt won as O {o_wins} times, drew {o_draws} times, and lost {o_losses} times.')

AlphaZero won as X 5 times, drew 2 times, and lost 3 times.
It won as O 3 times, drew 3 times, and lost 4 times.
